In [13]:
import sys
import subprocess

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# Install packages if missing
try:
    import great_expectations as gx
except ImportError:
    install("great_expectations")
    import great_expectations as gx

try:
    import pandas as pd
except ImportError:
    install("pandas")
    import pandas as pd

# Sample DataFrame with missing values
df = pd.DataFrame({
    "age": [25, 30, None, 45],
    "name": ["Alice", "Bob", "Charlie", "David"]
})

context = gx.get_context()

# Define datasource config dictionary (Fluent style)
datasource_config = {
    "name": "my_pandas",
    "class_name": "Datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine"
    },
    "data_connectors": {
        "default_runtime_data_connector": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"]
        }
    }
}

# Add datasource by config dict only (avoid passing datasource= param)
if "my_pandas" not in [ds["name"] for ds in context.list_datasources()]:
    context.add_datasource(**datasource_config)

from great_expectations.core.batch import BatchRequest

# Create BatchRequest with runtime data (our dataframe)
batch_request = BatchRequest(
    datasource_name="my_pandas",
    data_connector_name="default_runtime_data_connector",
    data_asset_name="my_data_asset",
    runtime_parameters={"batch_data": df},
    batch_identifiers={"default_identifier_name": "default_id"},
)

# Create or load expectation suite & get validator
expectation_suite_name = "test_suite"
try:
    suite = context.get_expectation_suite(expectation_suite_name)
except gx.exceptions.DataContextError:
    suite = context.create_expectation_suite(expectation_suite_name)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

# Add an expectation: no null values in 'age'
validator.expect_column_values_to_not_be_null("age")

# Run validation
results = validator.validate()

# Print validation summary
print(results)

# Save expectation suite
validator.save_expectation_suite()

# Generate and open Data Docs (html report)
context.build_data_docs()
context.open_data_docs()

DataContextError: Datasource is not a FluentDatasource

In [10]:
import sys
import subprocess

# Install packages if missing
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import great_expectations as gx
except ImportError:
    install("great_expectations")
    import great_expectations as gx

try:
    import pandas as pd
except ImportError:
    install("pandas")
    import pandas as pd

# Create a sample dataframe
df = pd.DataFrame({
    "age": [25, 30, None, 45],
    "name": ["Alice", "Bob", "Charlie", "David"]
})

# Initialize Great Expectations context (Fluent API)
context = gx.get_context()

from great_expectations.datasource.fluent import PandasDatasource
from great_expectations.validator.validator import Validator

# Add a Pandas datasource - must provide name (FluentDatasource required)
if "my_pandas" not in context.list_datasources():
    context.add_datasource(
        name="my_pandas",
        datasource=PandasDatasource(name="my_pandas")
    )

# Create Validator with runtime batch (your DataFrame)
validator: Validator = context.sources["my_pandas"].get_validator(
    batch_request={
        "batch_data": df,
        "datasource_name": "my_pandas",
        "data_connector_name": "default_runtime_data_connector",
        "data_asset_name": "my_data_asset",
    },
    create_expectation_suite=True,
    expectation_suite_name="test_suite"
)

# Add an expectation (age column should not have nulls)
validator.expect_column_values_to_not_be_null("age")

# Validate and print results
results = validator.validate()
print(results)

# Save expectation suite to disk
validator.save_expectation_suite(discard_failed_expectations=False)

# Generate data docs locally
context.build_data_docs()
context.open_data_docs()

TypeError: Must either pass in an existing 'datasource' or individual constructor arguments (but not both)